In [24]:
import gradio as gr
import openai
from helpers import get_client_and_model, generate_tts, get_system_prompt
import warnings

TOOL_SUPPORTED_MODELS = {"GPT", "Claude", "Gemini"}

def supports_tools(model_choice):
    return model_choice in TOOL_SUPPORTED_MODELS

warnings.filterwarnings("ignore", category=DeprecationWarning, module="gradio")

In [ ]:
%pip install ddgs

In [26]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Search the web for current information on a topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    },
                    "num_results": {
                        "type": "integer",
                        "description": "Number of results to return (default 3)",
                        "default": 3
                    }
                },
                "required": ["query"]
            }
        }
    }
]

In [27]:
from ddgs import DDGS

def search_web(query: str, num_results: int = 3) -> dict:
    print("searching web with query: ", query)
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=num_results):
            results.append({
                "title": r["title"],
                "url": r["href"],
                "snippet": r["body"]
            })
    return {"query": query, "results": results}

In [28]:
import json

def handle_function_calls(tool_calls, messages):
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        if function_name == "search_web":
            query = arguments.get("query", "")
            num_results = arguments.get("num_results", 3)
            result = search_web(query, num_results)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
        else:
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": "Function not implemented."
            })
    return messages

def build_assistant_tool_message(reply, tool_calls):
    return {
        "role": "assistant",
        "content": reply or None,
        "tool_calls": [
            {
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments,
                },
            }
            for tc in tool_calls
        ],
    }

In [29]:
def _stream_reply(response, tool_calls_out=None):
    """Yield cumulative reply text, optionally collecting tool calls into tool_calls_out dict."""
    reply = ""
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.content:
            reply += delta.content
            yield reply
        if tool_calls_out is not None and delta.tool_calls:
            for tc in delta.tool_calls:
                if tc.index not in tool_calls_out:
                    tool_calls_out[tc.index] = tc
                else:
                    tool_calls_out[tc.index].function.arguments += tc.function.arguments


def chat_stream(history, model_choice):
    messages = [{"role": "system", "content": get_system_prompt()}]
    messages += [
        {"role": h["role"], "content": h["content"]}
        for h in history if isinstance(h.get("content"), str)
    ]

    client, model = get_client_and_model(model_choice)
    use_tools = supports_tools(model_choice)

    kwargs = dict(model=model, messages=messages, stream=True)
    if use_tools:
        kwargs["tools"] = tools

    tool_calls = {} if use_tools else None
    reply = ""
    for reply in _stream_reply(client.chat.completions.create(**kwargs), tool_calls):
        history.append({"role": "assistant", "content": reply})
        yield history, None
        history.pop()

    if tool_calls:
        collected = list(tool_calls.values())
        messages.append(build_assistant_tool_message(reply, collected))
        handle_function_calls(collected, messages)

        reply = ""
        for reply in _stream_reply(client.chat.completions.create(model=model, messages=messages, stream=True)):
            history.append({"role": "assistant", "content": reply})
            yield history, None
            history.pop()

    history.append({"role": "assistant", "content": reply})
    yield history, generate_tts(reply)

In [30]:
def process_audio_input(audio_filepath, history, model_choice):
    if audio_filepath is None:
        return

    history.append({"role": "user", "content": gr.Audio(audio_filepath)})

    with open(audio_filepath, "rb") as f:
        transcription = openai.audio.transcriptions.create(model="whisper-1", file=f)

    history.append({"role": "user", "content": transcription.text})
    yield history, None, None

    for updated_history, tts_audio in chat_stream(history, model_choice):
        yield updated_history, None, tts_audio

In [31]:
def process_text_input(message, history, model_choice):
    if not message.strip():
        return 
    history.append({"role": "user", "content": message})
    for updated_history, tts_audio in chat_stream(history, model_choice):
        yield "", updated_history, tts_audio

In [32]:
with gr.Blocks(title="Technical Q&A Assistant") as ui:
    gr.Markdown("# Technical Q&A Assistant")

    with gr.Row():
        # for gradio version < 6
        # chatbot = gr.Chatbot(height=500, type="messages")
        chatbot = gr.Chatbot(height=500)

    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:", scale=4)
        audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Voice Input", scale=1)
    
    with gr.Row():
        audio_output = gr.Audio(label="Response Audio", autoplay=True)

    with gr.Row():
        model_selector = gr.Dropdown(
            choices=["GPT", "Claude", "Gemini", "Llama"],
            value="GPT",
            label="Select Model",
            scale=1,
        )
        clear = gr.Button("Clear", scale=1)
    
    message.submit(
        process_text_input,
        inputs=[message, chatbot, model_selector],
        outputs=[message, chatbot, audio_output],
    )

    audio_input.stop_recording(
        process_audio_input,
        inputs=[audio_input, chatbot, model_selector],
        outputs=[chatbot, audio_input, audio_output],
    )

    clear.click(lambda: ([], None, None, ""), outputs=[chatbot, audio_output, audio_input, message])


In [ ]:
ui.launch(inbrowser=True, auth=("mrpeski", "mrpeski"))